In [1]:
from functools import reduce
import re
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.optimize import linear_sum_assignment
import warnings

warnings.filterwarnings('ignore')

In [2]:
events1 = pd.read_csv('новые_протесты_рус3.csv')
events2 = pd.read_csv('росстат_общий_new.csv')


events1['Date'] = pd.to_datetime(events1['Date'], errors='coerce')
events1['year'] = events1['Date'].dt.year  
events1['Event text'] = 1

corrections = {
    "Кемерово":        ("Кемеровская",     "Кемеровский",   32701000),
    "Пермь":           ("Пермский",        "Пермский",      57701000),
    "Петрозаводск":    ("Карелия",         "Петрозаводский",86701000),
    "Казань":          ("Татарстан",       "Казань",        92701000),
    "Владивосток":     ("Приморский",      "Владивостокский",5701000),
    "Ростов":          ("Ярославская",     "Ростовский",    78637000),
    "Нижний Новгород": ("Нижегородская",   "Нижний Новгород",22701000),
    "Орёл":            ("Орловская",       "Орёл",          54701000),
    "Якутск":          ("Саха",            "Якутск",        98701000),
    "Барнаул":         ("Алтайский",       "Барнаул",       1701000),
    "Челябинск":       ("Челябинская",     "Челябинский",   75701000),
    "Магас":           ("Ингушетия",       "Магас",         26701000),
    "Черкесск":        ("Карачаево-Черкесская","Черкесский",91701000),
    "Элиста":          ("Калмыкия",        "Элиста",        85701000),
    "Тула":            ("Тульская",        "Тула",          70701000),
    "Кострома":        ("Костромская",     "Кострома",      34701000),
    "Грозный":         ("Чеченская",       "Грозный",       96701000),
    "Саранск":         ("Мордовия",        "Саранск",       89701000),
    "Волчанск":        ("Свердловская",    "Волчанский",    65735000),
    "Сочи":            ("Краснодарский",   "Сочи",          3726000),
    "Сыктывкар":       ("Коми",            "Сыктывкар",     87701000),
    "Магнитогорск":    ("Челябинская",     "Магнитогорский",75738000),
    "Чита":            ("Забайкальский",   "Чита",          76701000),
    "Невьянск":        ("Свердловская",    "Невьянский",    65714000),
    "Кормиловка":      ("Омская",          "Кормиловский",  52623000),
    "Златоуст":        ("Челябинская",     "Златоустовский",75712000),
    "Вологда":         ("Вологодская",     "Вологда",       19701000),
    "Киров-Чепецк":    ("Кировская",       "Кирово-Чепецк", 33707000),
    "Ростов-на-Дону":  ("Ростовская",      "Ростов-на-Дону",60701000),
    "Реж":             ("Свердловская",    "Режевской",     65720000),
    "Качканар":        ("Свердловская",    "Качканарский",  65743000),
    "Нижний Тагил":    ("Свердловская",    "Нижний Тагил",  65751000),
    "Бакал":           ("Башкортостан",    "Бакалинский",   80607000),
}

# 3) Применяем коррекцию
for wrong_name, (new_region, new_municipality, new_oktmo) in corrections.items():
    mask = events1['municipality'] == wrong_name
    events1.loc[mask, 'region_name']  = new_region
    events1.loc[mask, 'municipality'] = new_municipality
    events1.loc[mask, 'oktmo']        = new_oktmo





df = events2.merge(
    events1[['region_name','municipality', 'year', '# protesters', 'Event text']],
    on=['municipality', 'region_name', 'year'],
    how='left'
)


df = df[df['region_name'] != 'Москва'].reset_index(drop=True)
df = df[df['region_name'] != 'Санкт-Петербург'].reset_index(drop=True)
# df = df[df['region_name'] != 'Московская'].reset_index(drop=True)
df.to_csv('общие_данные_2010t7.csv', index=False)
df

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,# protesters,Event text
0,NaN,28602100,0.000000,Тверская,2010.0,1.9,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
1,NaN,28602100,0.000000,Тверская,2012.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
2,NaN,28602100,0.000000,Тверская,2014.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
3,NaN,28602100,0.000000,Тверская,2015.0,0.0,0.0,100.0,300.0,0.000000,0.0,0.0,NaN,NaN
4,NaN,28602400,0.000000,Тверская,2010.0,14.3,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180968,имени Полины Осипенко,8637000,91.832956,Хабаровский,2022.0,0.0,79.6,0.0,0.0,54.528488,0.0,0.0,NaN,NaN
180969,имени Полины Осипенко,8637000,91.832956,Хабаровский,2023.0,0.0,80.3,0.0,0.0,54.432127,0.0,0.0,NaN,NaN
180970,имени Полины Осипенко,8637406,93.960350,Хабаровский,2007.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
180971,имени Полины Осипенко,8637406,93.960350,Хабаровский,2008.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN


In [3]:
events2 = pd.read_csv('общие_данные_2010t7.csv')
protests = events2[events2['Event text'] == 1]
protests_0910 = protests[protests['year'].isin([2011, 2012])].copy()

# Для каждого oktmo записываем минимальный год протеста в 2009–2010
protests_0910['fprotets_year'] = protests_0910.groupby('municipality')['year'].transform('min')

# Оставляем по одной строке на oktmo (колонка fprotets_year уже заполнена правильно)
protests1 = protests_0910.drop_duplicates(subset=['municipality']).reset_index(drop=True)



protests.to_csv('протесты_2010t7.csv', index=False)
protests1.to_csv('протесты2009_2013_2010t7.csv', index=False)
protests1

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,# protesters,Event text,fprotets_year
0,Абакан,95701000,79.594802,Хакасия,2012.0,2.10,0.00,180.0,1946.00,40.891433,100.000000,4.633579e+07,100.0,1.0,2012.0
1,Архангельск,11701000,0.000000,Архангельская,2011.0,538.20,2.94,2400.0,5900.00,0.000000,0.000000,8.539844e+07,200.0,1.0,2011.0
2,Астрахань,12701000,86.242222,Астраханская,2011.0,1655.90,38.00,2000.0,4000.00,39.121620,100.000000,9.790655e+07,0.0,1.0,2011.0
3,Барнаул,1701000,98.226081,Алтайский,2011.0,86.00,0.00,100.0,5400.00,41.532612,94.159013,1.787782e+08,0.0,1.0,2011.0
4,Белгород,14701000,96.058305,Белгородская,2011.0,139.10,0.00,0.0,100.00,0.000000,0.000000,8.244528e+07,0.0,1.0,2011.0
5,Благовещенск,10701000,84.788814,Амурская,2011.0,96.40,0.00,600.0,500.00,42.505979,97.643301,6.080975e+07,1.0,1.0,2011.0
6,Брянск,15701000,97.707806,Брянская,2011.0,98.20,0.00,1200.0,2500.00,40.492615,0.000000,7.933087e+07,0.0,1.0,2011.0
7,Владивостокский,5701000,96.447139,Приморский,2011.0,99.40,0.00,0.0,2568.50,42.424584,95.968640,1.555050e+08,100.0,1.0,2011.0
8,Владимир,17701000,48.991974,Владимирская,2012.0,55.60,34.16,0.0,500.00,46.436342,99.257368,8.130472e+07,0.0,1.0,2012.0
9,Волгоград,18701000,97.113306,Волгоградская,2011.0,352.10,30.01,545.1,19324.30,0.000000,0.000000,2.197124e+08,20.0,1.0,2011.0


In [4]:
lol = pd.read_csv("протесты_2010t7.csv") 
sobytia = pd.read_csv("общие_данные_2010t7.csv")

noprotestALL = sobytia[~sobytia['oktmo'].isin(lol['oktmo'])].reset_index(drop=True)
noprotestALL.to_csv('единицы_без_протестов_2010t7.csv', index=False)
noprotestALL

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,# protesters,Event text
0,NaN,28602100,0.000000,Тверская,2010.0,1.9,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
1,NaN,28602100,0.000000,Тверская,2012.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
2,NaN,28602100,0.000000,Тверская,2014.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
3,NaN,28602100,0.000000,Тверская,2015.0,0.0,0.0,100.0,300.0,0.000000,0.0,0.0,NaN,NaN
4,NaN,28602400,0.000000,Тверская,2010.0,14.3,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176476,имени Полины Осипенко,8637000,91.832956,Хабаровский,2022.0,0.0,79.6,0.0,0.0,54.528488,0.0,0.0,NaN,NaN
176477,имени Полины Осипенко,8637000,91.832956,Хабаровский,2023.0,0.0,80.3,0.0,0.0,54.432127,0.0,0.0,NaN,NaN
176478,имени Полины Осипенко,8637406,93.960350,Хабаровский,2007.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
176479,имени Полины Осипенко,8637406,93.960350,Хабаровский,2008.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN


In [ ]:
PRE_YEARS  = [2010]                   # препротестный период
POST_YEARS = [2011, 2012]       # протестный период
MIN_POST_EVENTS = 1
TARGET = 100


df = pd.read_csv("протесты_2010t7.csv" )
df['# protesters'] = pd.to_numeric(df['# protesters'], errors='coerce').fillna(0)
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')
df['event_flag'] = ((df.get('Event text', np.nan) == 1) | (df['# protesters'] > 0)).astype(int)

#  Агрегаты 
def aggregate_for_years(sub_years, prefix):
    sub = df[df['year'].isin(sub_years)].copy()
    agg = (
        sub.groupby('municipality', as_index=False)
           .agg(**{
               f'{prefix}_events': ('event_flag', 'sum'),
               f'{prefix}_sum':    ('# protesters', 'sum'),
               f'{prefix}_mean':   ('# protesters', 'mean')
           })
    )
    return agg

pre  = aggregate_for_years(PRE_YEARS,  'pre')
post = aggregate_for_years(POST_YEARS, 'post')
agg = pd.merge(pre, post, on='municipality', how='outer').fillna(0)

# Скор
agg['score'] = (
    0.50 * np.log((agg['post_mean'] + 1) / (agg['pre_mean'] + 1)) +
    0.30 * np.log1p(agg['post_sum']) -
    0.20 * np.log1p(agg['pre_sum'])
)

agg = agg[agg['post_events'] >= MIN_POST_EVENTS]



top_munis = (
    agg.sort_values('score', ascending=False)
       .head(TARGET)
       .loc[lambda x: x['score'] >= 0, 'municipality']
       .unique()
)


df_post = df[df['year'].isin(POST_YEARS)]
final_df = (
    df_post[df_post['municipality'].isin(top_munis)]
    .drop_duplicates(subset='municipality')
    .reset_index(drop=True)
)

print(f"Выбрано {final_df['municipality'].nunique()} муниципалитетов.")
final_df.to_csv('без_протестов2009_2010t7.csv', index=False)
final_df


Выбрано 39 муниципалитетов.


,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,# protesters,Event text,event_flag
0,Астрахань,12701000,86.242222,Астраханская,2011,1655.9,38.00,2000.0,4000.00,39.121620,100.000000,9.790655e+07,0.0,1.0,1
1,Белгород,14701000,96.058305,Белгородская,2011,139.1,0.00,0.0,100.00,0.000000,0.000000,8.244528e+07,0.0,1.0,1
2,Брянск,15701000,97.707806,Брянская,2011,98.2,0.00,1200.0,2500.00,40.492615,0.000000,7.933087e+07,0.0,1.0,1
3,Владимир,17701000,48.991974,Владимирская,2012,55.6,34.16,0.0,500.00,46.436342,99.257368,8.130472e+07,0.0,1.0,1
4,Волгоград,18701000,97.113306,Волгоградская,2011,352.1,30.01,545.1,19324.30,0.000000,0.000000,2.197124e+08,20.0,1.0,1
5,Вологда,19701000,0.000000,Вологодская,2011,266.0,0.00,67.0,1070.00,40.882997,48.770540,8.159991e+07,0.0,1.0,1
6,Воронеж,20701000,98.485943,Воронежская,2011,182.6,0.00,500.0,3500.00,40.936929,100.000000,2.188282e+08,0.0,1.0,1
7,Грозный,96701000,3.382850,Чеченская,2011,0.0,0.00,0.0,0.00,0.000000,0.000000,2.889121e+07,40.0,1.0,1
8,Екатеринбург,65701000,94.934784,Свердловская,2011,258.0,0.00,500.0,4800.00,0.000000,0.000000,5.690012e+08,0.0,1.0,1
9,Иваново,24701000,96.291908,Ивановская,2011,99.6,0.00,5600.0,2770.00,0.000000,0.000000,7.270481e+07,50.0,1.0,1


In [6]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)



df_protests = pd.read_csv("протесты2009_2013_2010t7.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t7.csv")
df_events_full = pd.read_csv("общие_данные_2010t7.csv")

required_years = {2010.0, 2013, 2014.0, 2015.0, 2016.0}

df_events = df_events[df_events['year'].isin([2011, 2012])].reset_index(drop=True)

features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[['region_name', 'municipality', 'oktmo', 'fprotets_year'] + features].reset_index(drop=True)
df_events = df_events[['region_name', 'municipality', 'oktmo'] + features].reset_index(drop=True)

print("Размер df_protests:", len(df_protests))
print("Размер df_events:", len(df_events))

combined = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()

if degenerate:
    print(f"Вырожденные признаки (дисперсия <= {threshold}): {degenerate}")
    features = [f for f in features if f not in degenerate]
    print(f"Оставшиеся признаки: {features}")
else:
    print("Нет вырожденных признаков.")

combined = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined.T)
cov_inv = np.linalg.inv(cov_matrix)

complete_event_oktmos = []
for oktmo in df_events['oktmo'].unique():
    if check_years_completeness(oktmo, df_events_full, required_years):
        complete_event_oktmos.append(oktmo)

df_events_complete = df_events[df_events['oktmo'].isin(complete_event_oktmos)].reset_index(drop=True)

print(f"После фильтрации по годам осталось {len(df_events_complete)} муниципалитетов событий")

n_protests = df_protests.shape[0]
n_events = df_events_complete.shape[0]
distance_matrix = np.zeros((n_protests, n_events))

for i in range(n_protests):
    x = df_protests.loc[i, features].values
    for j in range(n_events):
        y = df_events_complete.loc[j, features].values
        distance_matrix[i, j] = mahalanobis(x, y, cov_inv)

row_ind, col_ind = linear_sum_assignment(distance_matrix)

df_matched_protests = df_protests.iloc[row_ind].reset_index(drop=True)
df_matched_events = df_events_complete.iloc[col_ind].reset_index(drop=True)

df_matches = df_matched_protests.join(df_matched_events, lsuffix="_protest", rsuffix="_event")
df_matches["mahalanobis_distance"] = distance_matrix[row_ind, col_ind]

df_matches.to_csv('прототип_данных_2010t7.csv', index=False)
df_matches

Размер df_protests: 59
Размер df_events: 17988
Нет вырожденных признаков.
После фильтрации по годам осталось 5585 муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2012.0,2.10,0.00,180.0,1946.00,Ленинградская,Сосновское,41639444,0.00,0.00,180.0,2000.0,0.013590
1,Архангельская,Архангельск,11701000,2011.0,538.20,2.94,2400.0,5900.00,Ленинградская,Выборгский,41615000,156.70,0.00,2438.0,6650.0,0.332689
2,Астраханская,Астрахань,12701000,2011.0,1655.90,38.00,2000.0,4000.00,Марий,Медведевский,88628000,279.50,33.88,1800.0,2427.0,0.887313
3,Алтайский,Барнаул,1701000,2011.0,86.00,0.00,100.0,5400.00,Тульская,Новомосковск,70724000,80.10,0.00,100.0,5460.0,0.015439
4,Белгородская,Белгород,14701000,2011.0,139.10,0.00,0.0,100.00,Тверская,Спировский,28652000,128.70,0.00,0.0,80.0,0.007595
5,Амурская,Благовещенск,10701000,2011.0,96.40,0.00,600.0,500.00,Пермский,Двуреченское,57646405,0.00,0.00,605.0,415.0,0.058329
6,Брянская,Брянск,15701000,2011.0,98.20,0.00,1200.0,2500.00,Оренбургская,Тоцкий,53652000,157.30,0.00,1200.0,2500.0,0.033019
7,Приморский,Владивостокский,5701000,2011.0,99.40,0.00,0.0,2568.50,Оренбургская,Соль-Илецкий,53644000,90.44,0.00,0.0,2500.0,0.017767
8,Владимирская,Владимир,17701000,2012.0,55.60,34.16,0.0,500.00,Забайкальский,Хилокский,76647000,19.20,34.18,0.0,500.0,0.020353
9,Волгоградская,Волгоград,18701000,2011.0,352.10,30.01,545.1,19324.30,Омская,Марьяновский,52630000,54.20,25.38,500.0,19000.0,0.288205


In [7]:
df_2012 = pd.read_csv('прототип_данных_2010t7.csv')
df_2011 = pd.read_csv('без_протестов2009_2010t7.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)


merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 
    'region_name_protest', 
    'region_name_event','fprotets_year'
]]


from2012.to_csv('связка_махаланобис_2010t7.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_protest,region_name_event,fprotets_year
0,Астрахань,12701000,1655.9,38.00,2000.0,4000.00,Медведевский,88628000,279.50,33.88,1800.0,2427.0,Астраханская,Марий,2011.0
1,Белгород,14701000,139.1,0.00,0.0,100.00,Спировский,28652000,128.70,0.00,0.0,80.0,Белгородская,Тверская,2011.0
2,Брянск,15701000,98.2,0.00,1200.0,2500.00,Тоцкий,53652000,157.30,0.00,1200.0,2500.0,Брянская,Оренбургская,2011.0
3,Владимир,17701000,55.6,34.16,0.0,500.00,Хилокский,76647000,19.20,34.18,0.0,500.0,Владимирская,Забайкальский,2012.0
4,Волгоград,18701000,352.1,30.01,545.1,19324.30,Марьяновский,52630000,54.20,25.38,500.0,19000.0,Волгоградская,Омская,2011.0
5,Вологда,19701000,266.0,0.00,67.0,1070.00,Копейский,75728000,263.70,0.00,100.0,800.0,Вологодская,Челябинская,2011.0
6,Воронеж,20701000,182.6,0.00,500.0,3500.00,Рузаевка,89643101,0.00,0.00,500.0,3400.0,Воронежская,Мордовия,2011.0
7,Грозный,96701000,0.0,0.00,0.0,0.00,Агульский,82601000,0.00,0.00,0.0,0.0,Чеченская,Дагестан,2011.0
8,Екатеринбург,65701000,258.0,0.00,500.0,4800.00,Акбулакский,53605000,36.80,0.00,500.0,4880.0,Свердловская,Оренбургская,2011.0
9,Иваново,24701000,99.6,0.00,5600.0,2770.00,Калязинский,28622000,15.80,0.00,4600.0,330.0,Ивановская,Тверская,2011.0


In [8]:
df = pd.read_csv('общие_данные_2010t7.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)',  'region_name',
        'Процент (рус)',
        ]]
df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])



# Фильтруем по годам 2011, 2013, 2014, 2015
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016])]
events = df
from2012 = pd.read_csv('связка_махаланобис_2010t7.csv')


final = pd.DataFrame()
for _, link_row in from2012.iterrows():
    

    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    

    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    

    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2011, 2012]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t7.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,Процент (рус)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,Процент (рус)_event,fprotets_year
0,2008.0,Астрахань,12701000,1762.1,6.39,420.0,6906.0,Астраханская,86.242222,Медведевский,88628000.0,42.40,0.00,110.0,508.0,Марий,49.337173,NaN
1,2009.0,Астрахань,12701000,1739.0,1.08,8230.0,1200.0,Астраханская,86.242222,Медведевский,88628000.0,36.32,1.01,140.0,1434.0,Марий,49.337173,NaN
2,2010.0,Астрахань,12701000,1672.9,91.15,6600.0,8800.0,Астраханская,86.242222,Медведевский,88628000.0,46.70,0.00,57.0,1694.0,Марий,49.337173,NaN
3,2011.0,Астрахань,12701000,1655.9,38.00,2000.0,4000.0,Астраханская,86.242222,Медведевский,88628000.0,215.10,19.79,52.0,1690.0,Марий,49.337173,2011.0
4,2012.0,Астрахань,12701000,1647.4,38.00,6290.0,18033.0,Астраханская,86.242222,Медведевский,88628000.0,279.50,33.88,1800.0,2427.0,Марий,49.337173,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,2012.0,Ярославль,78701000,283.1,25.42,3200.0,12560.0,Ярославская,0.000000,Марковское,19620444.0,0.00,0.00,7200.0,13800.0,Вологодская,0.000000,2011.0
343,2013.0,Ярославль,78701000,205.7,38.68,2160.0,16690.0,Ярославская,0.000000,Марковское,19620444.0,0.70,0.00,7200.0,13800.0,Вологодская,0.000000,NaN
344,2014.0,Ярославль,78701000,204.9,30.86,10150.0,22830.0,Ярославская,0.000000,Марковское,19620444.0,0.30,0.00,7200.0,13800.0,Вологодская,0.000000,NaN
345,2015.0,Ярославль,78701000,0.0,39.53,3500.0,11000.0,Ярославская,0.000000,Марковское,19620444.0,0.00,0.00,100.0,0.0,Вологодская,0.000000,NaN


ВТОРОЙ МАЧИНГ

In [9]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)

def matching_with_year_completeness(df_protests, df_events, df_events_full, features, required_years, cov_inv):
    matches_list = []
    
    for region in df_protests['region_name'].unique():
        sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
        sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
        
        complete_event_munis = []
        for _, row in sub_events.iterrows():
            if check_years_completeness(row['oktmo'], df_events_full, required_years):
                complete_event_munis.append(row['oktmo'])
        
        sub_events_complete = sub_events[sub_events['oktmo'].isin(complete_event_munis)].reset_index(drop=True)
        
        if len(sub_events_complete) == 0:
            print(f"В регионе {region} нет муниципалитетов событий с полными данными за все годы")
            continue
        
        n_p = sub_protests.shape[0]
        n_e = sub_events_complete.shape[0]
        dist_mat = np.zeros((n_p, n_e))
        
        for i in range(n_p):
            x = sub_protests.loc[i, features].values
            for j in range(n_e):
                y = sub_events_complete.loc[j, features].values
                dist_mat[i, j] = mahalanobis(x, y, cov_inv)
        
        row_ind, col_ind = linear_sum_assignment(dist_mat)
        
        matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
        matched_e = sub_events_complete.iloc[col_ind].reset_index(drop=True)
        matched = matched_p.join(
            matched_e,
            lsuffix="_protest",
            rsuffix="_event"
        )
        matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
        matches_list.append(matched)
    
    return matches_list

df_protests = pd.read_csv("протесты2009_2013_2010t7.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t7.csv")
df_events_full = pd.read_csv("общие_данные_2010t7.csv")

required_years = {2010.0, 2013, 2014.0, 2015.0, 2016.0}
df_events = df_events[df_events['year'].isin([2011, 2012])].reset_index(drop=True)


features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[
    ['region_name', 'municipality', 'oktmo', 'fprotets_year'] + features
].reset_index(drop=True)
df_events = df_events[
    ['region_name', 'municipality', 'oktmo'] + features
].reset_index(drop=True)

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

matches_list = matching_with_year_completeness(
    df_protests, df_events, df_events_full, features, required_years, cov_inv
)


df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2010t7M.csv', index=False)
df_matches
    


Вырожденных признаков нет.
В регионе Чеченская нет муниципалитетов событий с полными данными за все годы
В регионе Магаданская нет муниципалитетов событий с полными данными за все годы


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2012.0,2.10,0.00,180.0,1946.00,Хакасия,Черногорск,95715000,104.20,0.00,374.0,2101.0,0.258568
1,Архангельская,Архангельск,11701000,2011.0,538.20,2.94,2400.0,5900.00,Архангельская,Коряжма,11708000,1.00,0.00,1163.0,898.7,1.884253
2,Астраханская,Астрахань,12701000,2011.0,1655.90,38.00,2000.0,4000.00,Астраханская,Наримановский,12640000,26.70,41.40,200.0,2750.0,2.522906
3,Алтайский,Барнаул,1701000,2011.0,86.00,0.00,100.0,5400.00,Алтайский,Родинский,1636000,4.70,0.00,0.0,5200.0,0.139962
4,Белгородская,Белгород,14701000,2011.0,139.10,0.00,0.0,100.00,Белгородская,Алексеевский,14605000,28.40,0.00,300.0,0.0,0.406374
5,Амурская,Благовещенск,10701000,2011.0,96.40,0.00,600.0,500.00,Амурская,Тында,10732000,140.70,0.00,500.0,400.0,0.132510
6,Брянская,Брянск,15701000,2011.0,98.20,0.00,1200.0,2500.00,Брянская,Карачевский,15624000,10.00,0.00,400.0,1700.0,1.041503
7,Приморский,Владивостокский,5701000,2011.0,99.40,0.00,0.0,2568.50,Приморский,Михайловский,5620000,1.19,0.00,0.0,2544.0,0.055091
8,Владимирская,Владимир,17701000,2012.0,55.60,34.16,0.0,500.00,Владимирская,Округ,17735000,59.80,27.52,90.0,820.0,0.365779
9,Волгоградская,Волгоград,18701000,2011.0,352.10,30.01,545.1,19324.30,Волгоградская,Старополтавский,18652000,0.10,0.00,0.0,21900.0,1.886376


In [10]:
df_2012 = pd.read_csv('прототип_данных_2010t7M.csv')
df_2011 = pd.read_csv('без_протестов2009_2010t7.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 
    'region_name_protest', 
    'region_name_event', 'fprotets_year'
]]


from2012.to_csv('связка_махаланобис_2010t7М.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_protest,region_name_event,fprotets_year
0,Астрахань,12701000,1655.9,38.00,2000.0,4000.00,Наримановский,12640000,26.7,41.40,200.0,2750.0,Астраханская,Астраханская,2011.0
1,Белгород,14701000,139.1,0.00,0.0,100.00,Алексеевский,14605000,28.4,0.00,300.0,0.0,Белгородская,Белгородская,2011.0
2,Брянск,15701000,98.2,0.00,1200.0,2500.00,Карачевский,15624000,10.0,0.00,400.0,1700.0,Брянская,Брянская,2011.0
3,Владимир,17701000,55.6,34.16,0.0,500.00,Округ,17735000,59.8,27.52,90.0,820.0,Владимирская,Владимирская,2012.0
4,Волгоград,18701000,352.1,30.01,545.1,19324.30,Старополтавский,18652000,0.1,0.00,0.0,21900.0,Волгоградская,Волгоградская,2011.0
5,Вологда,19701000,266.0,0.00,67.0,1070.00,Тотемский,19646000,49.0,0.00,50.0,1000.0,Вологодская,Вологодская,2011.0
6,Воронеж,20701000,182.6,0.00,500.0,3500.00,Панинский,20635000,4.4,0.00,400.0,5000.0,Воронежская,Воронежская,2011.0
7,Екатеринбург,65701000,258.0,0.00,500.0,4800.00,Туринский,65726000,41.0,0.00,300.0,3800.0,Свердловская,Свердловская,2011.0
8,Иваново,24701000,99.6,0.00,5600.0,2770.00,Кохма,24706000,18.0,0.00,1936.0,602.0,Ивановская,Ивановская,2011.0
9,Ижевск,94701000,44.3,64.61,600.0,2000.00,Глазовский,94610000,5.3,63.97,532.0,1430.0,Удмуртская,Удмуртская,2011.0


In [11]:
df = pd.read_csv('общие_данные_2010t7.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)',  'region_name',
        'Процент (рус)',
        ]]
df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])


df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016])]
events = df
from2012 = pd.read_csv('связка_махаланобис_2010t7М.csv')


final = pd.DataFrame()
for _, link_row in from2012.iterrows():
    

    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    

    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    

    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2011, 2012]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t7М.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,Процент (рус)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,Процент (рус)_event,fprotets_year
0,2008.0,Астрахань,12701000,1762.1,6.39,420.0,6906.0,Астраханская,86.242222,Наримановский,12640000.0,26.90,0.00,0.0,1350.0,Астраханская,47.925996,NaN
1,2009.0,Астрахань,12701000,1739.0,1.08,8230.0,1200.0,Астраханская,86.242222,Наримановский,12640000.0,12.69,0.00,100.0,1100.0,Астраханская,47.925996,NaN
2,2010.0,Астрахань,12701000,1672.9,91.15,6600.0,8800.0,Астраханская,86.242222,Наримановский,12640000.0,26.70,0.00,100.0,1280.0,Астраханская,47.925996,NaN
3,2011.0,Астрахань,12701000,1655.9,38.00,2000.0,4000.0,Астраханская,86.242222,Наримановский,12640000.0,26.70,24.49,100.0,3230.0,Астраханская,47.925996,2011.0
4,2012.0,Астрахань,12701000,1647.4,38.00,6290.0,18033.0,Астраханская,86.242222,Наримановский,12640000.0,26.70,41.40,200.0,2750.0,Астраханская,47.925996,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,2012.0,Южно-Сахалинск,64701000,140.8,12.75,200.0,1900.0,Сахалинская,77.693897,Корсаковский,64716000.0,9.20,99.58,0.0,1100.0,Сахалинская,82.107766,2011.0
319,2013.0,Южно-Сахалинск,64701000,135.9,13.87,2400.0,8200.0,Сахалинская,77.693897,Корсаковский,64716000.0,27.20,99.58,0.0,700.0,Сахалинская,82.107766,NaN
320,2014.0,Южно-Сахалинск,64701000,244.3,17.73,2900.0,3000.0,Сахалинская,77.693897,Корсаковский,64716000.0,10.50,6.24,0.0,700.0,Сахалинская,82.107766,NaN
321,2015.0,Южно-Сахалинск,64701000,0.0,19.54,1700.0,5800.0,Сахалинская,77.693897,Корсаковский,64716000.0,0.00,7.30,0.0,36800.0,Сахалинская,82.107766,NaN


In [15]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)


df_protests = pd.read_csv("протесты2009_2013_2010t7.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t7.csv")
df_events_full = pd.read_csv("общие_данные_2010t7.csv")

required_years = {2010.0, 2013, 2014.0, 2015.0, 2016.0}


df_events = df_events[df_events['year'].isin([2011, 2012])].reset_index(drop=True)

features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[['region_name', 'municipality', 'oktmo', 'fprotets_year'] + features].reset_index(drop=True)
df_events = df_events[['region_name', 'municipality', 'oktmo'] + features].reset_index(drop=True)


exclude_dict = {
    'Брянск': ['Брянский', 'Сельцо', 'Фокино', 'Титовское'],
    'Вологда': ['Вологодский'],
    'Воронеж': ['Хохольский', 'Семилукский', 'Рамонский', 'Новоусманский', 'Каширский'],
    'Казань': ['Лаишевский', 'Пестречинский', 'Высокогорский', 'Зеленодольский', 'Верхнеуслонский'],
    'Краснодар': ['Динский', 'Красноармейский', 'Тахтамукайский'],
    'Омск': ['Омский'],
    'Тамбов': ['Тамбовский', 'Котовск'],
    'Тула': ['Щекинский', 'Дубенский', 'Алексин', 'Ясногорский', 'Веневский', 'Киреевский', 'Ленинский'],
    'Ярославль': ['Ярославский'],
    'Владивостокский': ['Хасанский', 'Надеждинский', 'Артемовский', 'Шкотовский', 'Большой Камень', 'Фокино'],
    'Южно-Сахалинск': ['Корсаковский', 'Анивский', 'Холмский', 'Долинский'],
    'Магадан': ['Ольский', 'Хасынский'],
    'Саранск': ['Рузаевский', 'Лямбирский', 'Кочкуровский'],
    'Тюмень': ['Тюменский', 'Нижнетавдинский'],
    'Липецк': ['Грязинский', 'Липецкий', 'Добровский'],
    'Астрахань': ['Приволжский', 'Икрянинский', 'Наримановский', 'Харабали', 'Верхний'],
    'Владимир': ['Собинский', 'Судогодский', 'Камешковский', 'Суздальский'],
    'Екатеринбург': ['Первоуральск', 'Дегтярск', 'Полевской', 'Сысертский', 'Белоярский', 'Березовский', 'Верхняя Пышма'],
    'Ижевск': ['Завьяловский', 'Камское'],
    'Киров': ['Оничевский', 'Орловский', 'Юрьянский', 'Слободской', 'Кирово-Чепецкий'],
    'Курган': ['Кетовский', 'Варгашинский', 'Каргапольский'],
    'Нальчик': ['Чегемский', 'Черекский'],
    'Самара': ['Волжский', 'Новокуйбышевск'],
    'Ставрополь': ['Шпаковский'],
    'Уфа': ['Уфимский', 'Кармаскалинский', 'Иглинский', 'Чишминский'],
    'Махачкала': ['Карабудахкентский', 'Каспийск', 'Буйнакский', 'Кумторкалинский', 'Новолакский'],
    'Саратов': ['Саратовский', 'Энгельсский', 'Марксовский', 'Энгельс'],
    'Томск': ['Томский'],
    'Ульяновск': ['Ульяновский', 'Новоульяновск', 'Чердаклинский'],
    'Волгоград': ['Городищенский', 'Дубовский', 'Среднеахтубинский', 'Светлоярский', 'Калачевский'],
    'Магас': ['Назрановский', 'Пригородный'],
    'Мурманск': ['Североморск', 'Кольский'],
    'Новосибирск': ['Новосибирский', 'Кольцово', 'Бердск', 'Обь'],
    'Петрозаводский': ['Прионежский'],
    'Ростовский': ['Борисоглебский', 'Переяславь-Залесский', 'Юрьев-Польский', 'Ильинский', 'Гаврилов-Ямский'],
    'Рязань': ['Рязанский'],
    'Смоленск': ['Смоленский'],
    'Чебоксары': ['Чебоксарский', 'Новочебоксарск', 'Звениговский'],
    'Челябинский': ['Сосновский', 'Копейский', 'Красноармейский'],
    'Грозный': ['Урус-Мартановский', 'Грозненский', 'Аргун'],
    'Белгород': ['Белгородский'],
    'Иваново': ['Ивановский', 'Кохма'],
    'Пенза': ['Заречный', 'Пензенский', 'Бессоновский'],

}


excluded_pairs = set()
for protest, events in exclude_dict.items():
    for event in events:
        excluded_pairs.add((protest, event))

# Проверка на вырожденные признаки
combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

# Matching внутри каждого региона с усиленной проверкой ограничений
matches_list = []

for region in df_protests['region_name'].unique():
    sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
    sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
    
    # Фильтрация событий: полнота данных + проверка на исключения
    valid_events = []
    for _, event_row in sub_events.iterrows():
        event_name = event_row['municipality']
        oktmo = event_row['oktmo']
        
        # Проверка полноты данных
        has_full_data = check_years_completeness(oktmo, df_events_full, required_years)
        
        # Проверка, что нет ни одного протеста, для которого это событие запрещено
        is_allowed = all(
            (protest_name, event_name) not in excluded_pairs
            for protest_name in sub_protests['municipality']
        )
        
        if has_full_data and is_allowed:
            valid_events.append(event_row)
    
    if not valid_events:
        print(f"В регионе {region} нет подходящих муниципалитетов событий")
        continue
    
    sub_events_valid = pd.DataFrame(valid_events).reset_index(drop=True)
    
    n_p = sub_protests.shape[0]
    n_e = sub_events_valid.shape[0]
    
    if n_e == 0:
        continue
    
    dist_mat = np.zeros((n_p, n_e))
    
    # Заполнение матрицы расстояний
    for i in range(n_p):
        x = sub_protests.loc[i, features].values
        for j in range(n_e):
            y = sub_events_valid.loc[j, features].values
            dist_mat[i, j] = mahalanobis(x, y, cov_inv)
    
    # Венгерский алгоритм
    row_ind, col_ind = linear_sum_assignment(dist_mat)
    
    # Сбор результатов
    matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
    matched_e = sub_events_valid.iloc[col_ind].reset_index(drop=True)
    
    matched = matched_p.join(
        matched_e,
        lsuffix="_protest",
        rsuffix="_event"
    )
    matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
    matches_list.append(matched)




df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2010t7M2.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Чеченская нет подходящих муниципалитетов событий
В регионе Магаданская нет подходящих муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2012.0,2.10,0.00,180.0,1946.00,Хакасия,Черногорск,95715000,104.20,0.00,374.0,2101.0,0.258568
1,Архангельская,Архангельск,11701000,2011.0,538.20,2.94,2400.0,5900.00,Архангельская,Коряжма,11708000,1.00,0.00,1163.0,898.7,1.884253
2,Астраханская,Астрахань,12701000,2011.0,1655.90,38.00,2000.0,4000.00,Астраханская,Лиманский,12635000,6.90,24.70,0.0,2920.0,2.821160
3,Алтайский,Барнаул,1701000,2011.0,86.00,0.00,100.0,5400.00,Алтайский,Родинский,1636000,4.70,0.00,0.0,5200.0,0.139962
4,Белгородская,Белгород,14701000,2011.0,139.10,0.00,0.0,100.00,Белгородская,Алексеевский,14605000,28.40,0.00,300.0,0.0,0.406374
5,Амурская,Благовещенск,10701000,2011.0,96.40,0.00,600.0,500.00,Амурская,Тында,10732000,140.70,0.00,500.0,400.0,0.132510
6,Брянская,Брянск,15701000,2011.0,98.20,0.00,1200.0,2500.00,Брянская,Карачевский,15624000,10.00,0.00,400.0,1700.0,1.041503
7,Приморский,Владивостокский,5701000,2011.0,99.40,0.00,0.0,2568.50,Приморский,Михайловский,5620000,1.19,0.00,0.0,2544.0,0.055091
8,Владимирская,Владимир,17701000,2012.0,55.60,34.16,0.0,500.00,Владимирская,Округ,17735000,59.80,27.52,90.0,820.0,0.365779
9,Волгоградская,Волгоград,18701000,2011.0,352.10,30.01,545.1,19324.30,Волгоградская,Старополтавский,18652000,0.10,0.00,0.0,21900.0,1.886376


In [16]:
df_2012 = pd.read_csv('прототип_данных_2010t7M2.csv')
df_2011 = pd.read_csv('без_протестов2009_2010t7.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 
    'region_name_protest', 
    'region_name_event', 'fprotets_year'
]]


from2012.to_csv('связка_махаланобис_2010t7М2.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_protest,region_name_event,fprotets_year
0,Астрахань,12701000,1655.9,38.00,2000.0,4000.00,Лиманский,12635000,6.9,24.70,0.0,2920.0,Астраханская,Астраханская,2011.0
1,Белгород,14701000,139.1,0.00,0.0,100.00,Алексеевский,14605000,28.4,0.00,300.0,0.0,Белгородская,Белгородская,2011.0
2,Брянск,15701000,98.2,0.00,1200.0,2500.00,Карачевский,15624000,10.0,0.00,400.0,1700.0,Брянская,Брянская,2011.0
3,Владимир,17701000,55.6,34.16,0.0,500.00,Округ,17735000,59.8,27.52,90.0,820.0,Владимирская,Владимирская,2012.0
4,Волгоград,18701000,352.1,30.01,545.1,19324.30,Старополтавский,18652000,0.1,0.00,0.0,21900.0,Волгоградская,Волгоградская,2011.0
5,Вологда,19701000,266.0,0.00,67.0,1070.00,Тотемский,19646000,49.0,0.00,50.0,1000.0,Вологодская,Вологодская,2011.0
6,Воронеж,20701000,182.6,0.00,500.0,3500.00,Панинский,20635000,4.4,0.00,400.0,5000.0,Воронежская,Воронежская,2011.0
7,Екатеринбург,65701000,258.0,0.00,500.0,4800.00,Туринский,65726000,41.0,0.00,300.0,3800.0,Свердловская,Свердловская,2011.0
8,Иваново,24701000,99.6,0.00,5600.0,2770.00,Приволжский,24620000,40.1,99.71,3600.0,5400.0,Ивановская,Ивановская,2011.0
9,Ижевск,94701000,44.3,64.61,600.0,2000.00,Глазовский,94610000,5.3,63.97,532.0,1430.0,Удмуртская,Удмуртская,2011.0


In [17]:
df = pd.read_csv('общие_данные_2010t7.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)',  'region_name',
        'Процент (рус)',
        ]]
df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])


df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016])]
events = df
from2012 = pd.read_csv('связка_махаланобис_2010t7М2.csv')


final = pd.DataFrame()
for _, link_row in from2012.iterrows():
    

    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    

    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    

    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2011, 2012]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t7М2.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,Процент (рус)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,Процент (рус)_event,fprotets_year
0,2008.0,Астрахань,12701000,1762.1,6.39,420.0,6906.0,Астраханская,86.242222,Лиманский,12635000.0,13.30,0.00,300.0,6250.0,Астраханская,83.872632,NaN
1,2009.0,Астрахань,12701000,1739.0,1.08,8230.0,1200.0,Астраханская,86.242222,Лиманский,12635000.0,6.95,0.00,0.0,16170.0,Астраханская,83.872632,NaN
2,2010.0,Астрахань,12701000,1672.9,91.15,6600.0,8800.0,Астраханская,86.242222,Лиманский,12635000.0,7.00,0.00,0.0,4950.0,Астраханская,83.872632,NaN
3,2011.0,Астрахань,12701000,1655.9,38.00,2000.0,4000.0,Астраханская,86.242222,Лиманский,12635000.0,7.00,82.38,0.0,5430.0,Астраханская,83.872632,2011.0
4,2012.0,Астрахань,12701000,1647.4,38.00,6290.0,18033.0,Астраханская,86.242222,Лиманский,12635000.0,6.90,24.70,0.0,2920.0,Астраханская,83.872632,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,2012.0,Южно-Сахалинск,64701000,140.8,12.75,200.0,1900.0,Сахалинская,77.693897,Северо-Курильский,64743000.0,17.10,99.68,400.0,100.0,Сахалинская,78.764940,2011.0
319,2013.0,Южно-Сахалинск,64701000,135.9,13.87,2400.0,8200.0,Сахалинская,77.693897,Северо-Курильский,64743000.0,22.00,0.00,100.0,4500.0,Сахалинская,78.764940,NaN
320,2014.0,Южно-Сахалинск,64701000,244.3,17.73,2900.0,3000.0,Сахалинская,77.693897,Северо-Курильский,64743000.0,19.70,0.00,0.0,0.0,Сахалинская,78.764940,NaN
321,2015.0,Южно-Сахалинск,64701000,0.0,19.54,1700.0,5800.0,Сахалинская,77.693897,Северо-Курильский,64743000.0,0.00,0.00,0.0,0.0,Сахалинская,78.764940,NaN
